In [3]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import re
from unidecode import unidecode
import pickle

# Analyse de 3 modèles de NLP

## On veut comparer les performances de 3 modèles pour prédire l'orientation de commentaires sur Amazon

## Scraping Commentaires Amazon

In [ ]:
head={
"Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"}

In [ ]:
note=[]
texte=[]
for i in range(1,100000):
    # Pause avant chaque requete d'un temps arbitraire entre 0.5 et 1.5 s
    time.sleep(random.uniform(0.5,1.5))
    url='https://www.amazon.fr/quatre-accords-tolt%C3%A8ques-libert%C3%A9-personnelle/product-reviews/2889116549/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(str(i))
    #url="https://www.amazon.fr/Apple-AirPods-bo%C3%AEtier-charge-Dernier/product-reviews/B07PZR3PVB/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
    r=requests.get(url,headers=head)
    # Si le retour request est diff de 200 ==> pb
    print(url,r.status_code)
    soup=BeautifulSoup(r.content)
    
    tmp_note=[]
    tmp_texte=[]
    for elem in soup.findAll('i',attrs={"data-hook":"review-star-rating"}):
        tmp_note.append(elem.text.split(',')[0])
        

    for elem in soup.findAll('span',attrs={"data-hook":"review-body"}):
        tmp_texte.append(elem.text.strip())
        

    if(len(tmp_note)==len(tmp_texte)):
      note.extend(tmp_note)
      texte.extend(tmp_texte)
    else:
      break

    print("iteration : ",i,"/ longueur liste ",len(note),len(texte))
    
    '''if (len(note),len(texte)) != (i*10,i*10):
        m=min(len(note),len(texte))
        note=note[:m]
        texte=texte[:m]
        print("break")
        break'''


https://www.amazon.fr/quatre-accords-tolt%C3%A8ques-libert%C3%A9-personnelle/product-reviews/2889116549/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1 200
iteration :  1 / longueur liste  10 10
https://www.amazon.fr/quatre-accords-tolt%C3%A8ques-libert%C3%A9-personnelle/product-reviews/2889116549/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2 200
iteration :  2 / longueur liste  20 20
https://www.amazon.fr/quatre-accords-tolt%C3%A8ques-libert%C3%A9-personnelle/product-reviews/2889116549/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=3 200
iteration :  3 / longueur liste  30 30
https://www.amazon.fr/quatre-accords-tolt%C3%A8ques-libert%C3%A9-personnelle/product-reviews/2889116549/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=4 200
iteration :  4 / longueur liste  40 40
https://www.amazon.fr/quatre-accords-tolt%C3%A8ques-libert%C3%A9-personnelle/product-reviews

In [ ]:
df=pd.DataFrame({'commentaire':texte,'label':note})

df['label'].value_counts()

5    780
4    128
1     68
3     54
2     30
Name: label, dtype: int64

In [ ]:
df.shape

(5990, 2)

In [ ]:
df.to_csv('Tolteques',index=False)

### Nettoyage du Texte 

In [ ]:
df=pd.read_csv('Tolteques.csv')

In [ ]:
print(df.shape)
df.drop_duplicates(subset=['commentaire'],inplace=True)
df.shape

(1060, 2)


(1028, 2)

In [ ]:
seuil_mot=10
def nb_mot(texte):
    return len(texte.split(' '))

df['Longueur']=df['commentaire'].apply(nb_mot)
df=df[df['Longueur']>seuil_mot]

In [ ]:
df['label'].value_counts()

# Binarisation 0 (1,2,3) / 1 (4,5)
df.loc[df[df['label'].astype(int)<3].index,'label']=0
df.loc[df[df['label'].astype(int)>3].index,'label']=1
df=df[df['label']!=3]
# Je prends autant de texte positif que negatif (don j'identifie le label le moins present)
m=df['label'].value_counts().min()
Comments=pd.concat([df[df['label'].astype(int)==1].sample(m),df[df['label'].astype(int)==0].sample(m)])

In [ ]:
Comments['label'].value_counts()

1    42
0    42
Name: label, dtype: int64

In [ ]:
Comments

,commentaire,label,Longueur
540,Remise en questions intéressante sur notre faç...,1,31
572,Livre facile à lire et à comprendre. Reste plu...,1,17
226,"Facile à lire et, déjà dit moult fois mais bie...",1,24
982,Superbe entrée pour la construction d'un monde...,1,37
25,Au début j'étais un peu sceptique par l'aspect...,1,69
...,...,...,...
1021,"Je l'ai lu volontiers, et en effet il reprend ...",0,67
167,Je n’ai pas du tout accroché la façon d’écrire...,0,42
967,Si on enlève tout ce qui est du bon sens et qu...,0,59
931,Comme la plupart des livres sur le développeme...,0,127


#### Nettoyage spécifique pour de l’analyse de texte

- Stop words
- carcteres speciaux (dont ponctuation)
- normalisation (accents et minuscule)


In [ ]:
Comments.to_csv("comments.csv")

In [2]:
import pandas as pd
Comments=pd.read_csv("comments.csv")

In [4]:
def nettoyage(texte):
    #Je charge la liste de stop words
    with open('stop-w_fr.txt') as f:
        s_w=list(set(unidecode(f.read()).split('\n')))
    s_w.remove('tres')
    s_w.remove('pas')
    s_w.remove('ne')
    s_w.remove('bon')
    s_w.remove('bien')
    #s_w.remove('mal')
    
    tmp=[]
    # Normalisation : accents et minuscule
    texte=unidecode(texte.lower())
    p='[a-z]{2,}'
    for mot in re.findall(p,texte):
        if mot in s_w:
            continue
        else:
            tmp.append(mot)
    return ' '.join(tmp)



In [5]:
negatif=[nettoyage(elem) for elem in Comments[Comments['label']==0]['commentaire']]
positif=[nettoyage(elem) for elem in Comments[Comments['label']==1]['commentaire']]

## TFIDF


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
#On précise ici le ngram range pour préciser la taille des séquences de mots que l'on utilse dans notre vectorizer
vectorizer = TfidfVectorizer(ngram_range=(2, 4))
X = vectorizer.fit_transform(Comments["commentaire"])
print(vectorizer.get_feature_names())

print(X.shape)

['aborde lecture', 'aborde lecture livre', 'aborde lecture livre rapidement', 'abscon livre', 'abscon livre demande', 'abscon livre demande lectures', 'absolu exemple', 'absolu exemple histoireem', 'absolu exemple histoireem ne', 'abstraits mal', 'abstraits mal rattacher', 'abstraits mal rattacher vie', 'accepte monde', 'accepte monde realites', 'accepte monde realites vie', 'accessible vrai', 'accessible vrai bcp', 'accessible vrai bcp aime', 'accessibles pratiques', 'accompagne maitre', 'accompagne maitre spirituel', 'accompagne maitre spirituel mettre', 'accord accord', 'accord accord tolteque', 'accord accord tolteque accord', 'accord cancer', 'accord cancer an', 'accord cancer an puissance', 'accord exemple', 'accord franchement', 'accord franchement pages', 'accord franchement pages mythe', 'accord impeccable', 'accord impeccable recommande', 'accord impeccable recommande recherche', 'accord ne', 'accord ne supposera', 'accord ne supposera pas', 'accord ruiz', 'accord ruiz esoter

In [45]:
Mat=pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
Mat

,aborde lecture,aborde lecture livre,aborde lecture livre rapidement,abscon livre,abscon livre demande,abscon livre demande lectures,absolu exemple,absolu exemple histoireem,absolu exemple histoireem ne,abstraits mal,...,vraiment pas succes,vraiment pas succes livre,vraiment tres,vraiment tres bien,what fur,what fur mesure,what fur mesure reel,wikipedia laissez,wikipedia laissez livre,wikipedia laissez livre etagere
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Classification TFIDF

In [46]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
clf = svm.LinearSVC()
y=Comments["label"]
clf.fit(X,y)  


from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, y, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.67 accuracy with a standard deviation of 0.06


### 0.67 accuracy with a standard deviation of 0.06

## Word2Vec

In [19]:

import spacy
# Chargement du modele français
nlp = spacy.load('fr_core_news_sm')

In [36]:
print(positif)

['remise questions interessante vivre contemporaine pas facile appliquer accords permettent bouleverser habitudes remise questions profitable', 'livre facile lire comprendre appliquer fabuleux conseils mr ruiz', 'facile lire moult bien vrai lit relit besoin recommande vivement', 'superbe entree construction monde meilleur construire bonheur participer bonheur humanite livre simple profond relecture', 'sceptique aspect religieux trouver livre continue livre excellent quatres accords difficiles appliquer permettent voir vie bon cote demande pratique changer progressivement vie livre recommande lire relire', 'lire attentivement appliquer accords avancer sereinement vie', 'bon livre probleme realiser accords', 'super livre facile lire souhaitent interesser developpement personnel message clair livre bien', 'livre clair intelligent essentiel chemin mieux offert lecteur aime pas importance style anglo saxon verites rabachees', 'parfait bible conserver guide quotidien puissions vivre reve ind

In [47]:
sentences=[x.split(" ") for x in Comments["commentaire"]]

In [51]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=sentences,hs=1,negative=0)

In [52]:
model.save("word2vec.model")

In [54]:
word2vec_df=pd.DataFrame(model.score(sentences, total_sentences=1000000, chunksize=100, queue_factor=2, report_delay=1))

In [55]:
word2vec_df

,0
0,-19.113998
1,-18.414871
2,-16.287033
3,-6.377422
4,-49.511497
...,...
79,-7.070462
80,-19.132271
81,-50.903664
82,-104.056946


In [56]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
clf = svm.LinearSVC()
y=Comments["label"]
clf.fit(word2vec_df,y)  


from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, word2vec_df, y, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

0.59 accuracy with a standard deviation of 0.12


### 0.59 accuracy with a standard deviation of 0.12

In [42]:
def nett(tex):
    # Conserver tous les caracteres sauf les speciaux ($¨#([)) et le chiffres
    p='[a-zA-Zéâûèàùîôêç\-.!?:]{1,}'
    
    # stop words
    with open('stop-w_fr.txt') as f:
        s_w=f.read().split('\n')
    s_w.remove('fait')
    
    
    
    # Laisser un peu de place autour de chaque separateur de phrase
    tex=tex.replace('!',' ! ').replace('.',' . ').replace('?',' ? ').replace(':',' : ')
    
    # lematizer ou stemmer
    tex=' '.join([elem.lemma_ for elem in nlp(tex,disable=['tagger', 'parser', 'ner'])])
    
    final=[]
    for mot in re.findall(p,tex):
        if mot.lower() not in s_w :
            final.append(mot.lower())


    return ' '.join(final).strip()

### Camembert

In [47]:
from IPython.display import Image
import gc
import numpy as np
import pandas as pd

In [1]:
import torch
from transformers import CamembertTokenizer,CamembertModel

tokenizer = CamembertTokenizer.from_pretrained('camembert/camembert-large')
model = CamembertModel.from_pretrained('camembert/camembert-large',output_hidden_states=True)

In [7]:
# 1)
input_ids = tokenizer.encode(positif[0],return_tensors='pt') 

# 2)
outputs = model(input_ids)
outputs[2][-4:]

(tensor([[[ 0.1082,  0.2615,  0.0994,  ...,  0.1266, -0.0500, -0.2383],
          [-0.3460,  0.2176, -0.1272,  ...,  0.1406, -0.0371, -0.2461],
          [-0.1364,  0.2826, -0.4707,  ..., -0.1067,  0.7043,  0.5065],
          ...,
          [-0.3521,  0.5399, -0.4337,  ...,  0.0222,  0.5779,  0.8215],
          [-0.0235, -0.0242,  0.0431,  ...,  0.0146, -0.0161, -0.0293],
          [-0.0231, -0.0243,  0.0446,  ...,  0.0152, -0.0164, -0.0300]]],
        grad_fn=<NativeLayerNormBackward>),
 tensor([[[-0.2115,  0.3425,  0.3733,  ...,  0.2257, -0.0157,  0.1408],
          [-0.0977,  0.1453, -0.3412,  ...,  0.1082, -0.0187,  0.0246],
          [-0.1728, -0.0213, -0.6117,  ..., -0.0316,  0.6748,  0.5547],
          ...,
          [-0.5288,  0.0494, -0.5013,  ...,  0.1276,  0.5644,  0.6705],
          [-0.0271, -0.0377,  0.0274,  ...,  0.0274,  0.0194, -0.0350],
          [-0.0274, -0.0384,  0.0280,  ...,  0.0281,  0.0196, -0.0355]]],
        grad_fn=<NativeLayerNormBackward>),
 tensor([[[-4.

In [10]:
import numpy as np
def feat(texte):
    input_ids = tokenizer.encode(texte, return_tensors='pt') 
    outputs = model(input_ids)
    
    l=[]
    for i in range(-1,-5,-1):
        l.append(pd.DataFrame(np.mean(outputs[2][i].squeeze(0).detach().numpy(),axis=0)).T)
    
    return pd.DataFrame(pd.concat(l,ignore_index=True).mean(axis=0)).T

from sklearn.metrics.pairwise import cosine_similarity

In [11]:
cosine_similarity(feat(positif[0]),feat(positif[1]))

array([[0.90660477]], dtype=float32)

## Featurisation Camembert  

In [14]:
Comments

,Unnamed: 0,commentaire,label,Longueur
0,540,Remise en questions intéressante sur notre faç...,1,31
1,572,Livre facile à lire et à comprendre. Reste plu...,1,17
2,226,"Facile à lire et, déjà dit moult fois mais bie...",1,24
3,982,Superbe entrée pour la construction d'un monde...,1,37
4,25,Au début j'étais un peu sceptique par l'aspect...,1,69
...,...,...,...,...
79,1021,"Je l'ai lu volontiers, et en effet il reprend ...",0,67
80,167,Je n’ai pas du tout accroché la façon d’écrire...,0,42
81,967,Si on enlève tout ce qui est du bon sens et qu...,0,59
82,931,Comme la plupart des livres sur le développeme...,0,127


In [18]:
Comments.describe()

,Unnamed: 0,label,Longueur
count,84.000000,84.000000,84.000000
mean,603.690476,0.500000,52.309524
std,338.492725,0.503003,79.186675
min,0.000000,0.000000,11.000000
25%,291.500000,0.000000,19.000000
50%,599.000000,0.500000,27.500000
75%,929.500000,1.000000,43.000000
max,1025.000000,1.000000,581.000000


In [20]:
Comments["commentaire"]=Comments["commentaire"].apply(nettoyage)

In [21]:
Comments

,Unnamed: 0,commentaire,label,Longueur
0,540,remise questions interessante vivre contempora...,1,31
1,572,livre facile lire comprendre appliquer fabuleu...,1,17
2,226,facile lire moult bien vrai lit relit besoin r...,1,24
3,982,superbe entree construction monde meilleur con...,1,37
4,25,sceptique aspect religieux trouver livre conti...,1,69
...,...,...,...,...
79,1021,lu volontiers reprend concepts communs ancienn...,0,67
80,167,pas accroche ecrire impression ne pas phase to...,0,42
81,967,enleve bon sens tiendrait pages livre resonne ...,0,59
82,931,livres developpement personnel transporte cour...,0,127


In [36]:
from tqdm import tqdm

inputs=[]
for elem in (Comments["commentaire"]):
    print(elem)
    inputs.append(feat(elem))
tmp=list(map(lambda elem: elem[0],(np.array(inputs))))

remise questions interessante vivre contemporaine pas facile appliquer accords permettent bouleverser habitudes remise questions profitable
livre facile lire comprendre appliquer fabuleux conseils mr ruiz
facile lire moult bien vrai lit relit besoin recommande vivement
superbe entree construction monde meilleur construire bonheur participer bonheur humanite livre simple profond relecture
sceptique aspect religieux trouver livre continue livre excellent quatres accords difficiles appliquer permettent voir vie bon cote demande pratique changer progressivement vie livre recommande lire relire
lire attentivement appliquer accords avancer sereinement vie
bon livre probleme realiser accords
super livre facile lire souhaitent interesser developpement personnel message clair livre bien
livre clair intelligent essentiel chemin mieux offert lecteur aime pas importance style anglo saxon verites rabachees
parfait bible conserver guide quotidien puissions vivre reve indefiniment guerrier spirituel


decevant auteur repete chose page explication alambique ne comprends vraiment pas succes livre
avoue titre intrigant accrocheur vois nombre lecteurs livre nombre lecteurs satisfaits livre panier reception surprise attendais ne demandez pas pave pages format livre poche pages surprise depasse dis mieux livre apporte lecture courte final partie lecture avide revelations cheminement personnel quadragenaire rime pas mal questionnement premieres pages obstacles dieu voix entendons tetes voix etres plans citations bible cadeau vient dieu prise doute demandant livre allait convenir fini convaincre emballage croyance message allais trouver messages substantiels apporte chose continue fini livre coup petite taille aide continuer lecture convenant allait pas prendre ne convenait pas partie lieux homme eleve naissance domestication apprentissage bien mal humain societe accords passes obtenir assentiment reconnaissance amour auteur arrivant conclusion accords ne fondes peurs malvenus pompent energ

In [41]:
inputs

[       0         1         2         3         4         5         6     \
 0 -0.098677  0.097899 -0.234988 -0.090113  0.040462  0.200902 -0.071933   
 
       7         8         9     ...      1014      1015     1016      1017  \
 0 -0.00399  0.066703  0.067062  ...  0.186164  0.082208  0.39618 -0.057935   
 
        1018      1019      1020      1021      1022      1023  
 0 -0.166702  0.100016  0.061093  0.209008  0.036793  0.106193  
 
 [1 rows x 1024 columns],
        0         1         2         3         4         5         6     \
 0 -0.031934 -0.114483  0.013333  0.118148  0.096389  0.130505 -0.073222   
 
      7         8         9     ...      1014      1015      1016     1017  \
 0 -0.1039 -0.045828 -0.042994  ...  0.201925 -0.019555  0.339409  0.04407   
 
        1018      1019      1020      1021      1022      1023  
 0 -0.132125  0.160427  0.156133  0.221446  0.059481  0.209803  
 
 [1 rows x 1024 columns],
        0         1        2         3         4         5

In [38]:
X=pd.DataFrame(tmp)
y=Comments['label']

In [42]:
X

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.098677,0.097899,-0.234988,-0.090113,0.040462,0.200902,-0.071933,-0.003990,0.066703,0.067062,...,0.186164,0.082208,0.396180,-0.057935,-0.166702,0.100016,0.061093,0.209008,0.036793,0.106193
1,-0.031934,-0.114483,0.013333,0.118148,0.096389,0.130505,-0.073222,-0.103900,-0.045828,-0.042994,...,0.201925,-0.019555,0.339409,0.044070,-0.132125,0.160427,0.156133,0.221446,0.059481,0.209803
2,-0.072364,0.244192,-0.090720,0.021808,0.094184,0.160669,-0.066175,-0.135242,-0.009338,-0.073838,...,0.257411,0.011786,0.615211,0.084578,-0.274376,0.187425,0.154403,0.274461,-0.148035,0.097222
3,-0.022584,0.132881,0.082987,-0.077622,-0.036784,0.258955,0.077048,0.055391,0.214594,-0.051583,...,0.000748,0.183673,0.762520,-0.085650,-0.253166,-0.000918,0.023169,0.534233,-0.090200,0.115082
4,-0.032511,0.096552,0.186293,-0.044306,0.069558,0.184519,0.055409,-0.106517,0.105811,-0.007097,...,0.549935,0.024404,1.055864,-0.020783,-0.214737,0.034652,0.145412,0.232756,-0.003778,0.250012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0.127840,0.256349,-0.050964,-0.089582,-0.003121,0.070729,-0.020103,0.056153,0.206381,-0.084909,...,0.057144,0.035309,0.757283,-0.017348,-0.113299,0.064745,0.022104,0.343085,0.069922,0.161932
80,0.056336,0.292211,-0.010021,-0.044750,-0.019294,0.138698,-0.074295,-0.066612,0.006502,-0.068118,...,0.273189,0.110959,0.669894,0.018571,-0.070853,0.015380,0.093312,0.337468,0.035538,0.077012
81,-0.013462,0.195254,-0.061641,-0.046965,0.086286,0.042798,-0.008175,-0.052453,0.107790,-0.024945,...,0.049688,0.021645,0.759525,0.057707,-0.166891,0.187181,0.089237,0.375356,0.014068,0.096136
82,0.037227,0.071607,0.033371,0.050299,0.036159,0.077464,-0.088602,0.023470,0.023988,0.001205,...,0.259283,0.087312,0.956634,0.057950,-0.237891,-0.056632,0.085848,0.312632,-0.001912,0.193812


### Classification Camembert avec SVM 

In [43]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
clf = svm.LinearSVC()
clf.fit(X, y)  


from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, y, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


#print("Train score : ",clf.score(Xtr, ytr))
#print("Test score : ",clf.score(Xte, yte))
#y_pred=clf.predict(Xte)
#confusion_matrix(yte, y_pred)

0.81 accuracy with a standard deviation of 0.09


### 0.81 accuracy with a standard deviation of 0.09

## Analyse des performances

| Modèles | TFIDF         | Word2VEC | Camembert|
| :--------------- |:---------------:| :-----:|----:
| Précision  |   .67        |  .59 | .81|
| Ecart Type  | .06            |   .12 | 0.09|
| Commentaire  | Mauvais résultat, le modèle n'est peut être pas adapté         |    Mauvais résultat, le modèle n'est peut être pas adapté | Bon résultat|

### Le camembert semble être le meilleur choix